# YOLO v8 model training

#### Import the base YOLO model

Check graphics card availability

In [ ]:
!nvidia-smi

Set HOME path for console interaction as the root of the repository

In [ ]:
import os
HOME = os.path.split(os.getcwd())[0]
print(HOME)

Install dependencies for model, including Torch, Ultralytics packages

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install ultralytics

Check YOLO package installation

In [ ]:
import ultralytics
ultralytics.checks()

Check Torch installation and CUDA acceleration availability

In [ ]:
# This is available for systems with nvidia GPUs to enable training accelaration with graphics cards:
# CUDA package: https://developer.nvidia.com/cuda-downloads

import torch
torch.cuda.is_available()

### Training with custom dataset

Training with client

In [ ]:
# The model is YOLOv8n, which is the most lightweight of all the YOLOv8 versions, with the least amount of params
# The dataset is manually tagged and downloaded through roboflow, the .yaml file is used to identify all the train/test/val datasets

%cd {HOME}

!yolo task=detect mode=train model={HOME}/yolov8n.pt data={HOME}/data/CowID.v7i.yolov8/data.yaml epochs=40 imgsz=640 plots=True

If the training isn't completed in one session, it can be continued using the following command

In [ ]:
# !yolo detect train resume model=C:\Users\arihs\Documents\Thesis\notebooks\runs\detect\train18\weights\last.pt

Display training results

In [ ]:
from IPython.display import Image, display

%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)

### Prediction using client

In [ ]:
# The console provide ways to predict videos, images, etc.

%cd {HOME}

!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.5 source={HOME}/data/test_4.mp4 save=True

### Prediction using Python

OpenCV is a popular framework to work with images

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
from ultralytics import YOLO
from PIL import Image
import os

HOME = os.path.split(os.getcwd())[0]

In [ ]:
# Load the model

model = YOLO(f"{HOME}/runs/detect/train/weights/best.pt")

# Make predictions based on images

results = model.predict(source=f"{HOME}/test_4_frames")

#### Create Siamese network dataset

In [ ]:
# Given a folder with images of cows, the model predicts the bounding boxes of the cows
# and expand_rectangle_to_square changes the shape of the prediction to a square
# then crop_to_square is used extract the cow from the image and save it

from PIL import Image

model = YOLO(f"{HOME}/runs/detect/train/weights/best.pt")

img_dir = f'{HOME}/test_4_frames/'

def expand_rectangle_to_square(x1, y1, x2, y2):
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    size = max(width, height)
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    new_x1 = center_x - size / 2
    new_y1 = center_y - size / 2
    new_x2 = center_x + size / 2
    new_y2 = center_y + size / 2
    return new_x1, new_y1, new_x2, new_y2

def crop_to_square(image_path, x1, y1, x2, y2):
    image = Image.open(image_path)
    cropped_image = image.crop((x1, y1, x2, y2))
    return cropped_image
    
    
for number in range(len(os.listdir(img_dir))-1):
    img_path = os.path.join(img_dir, f"frame{number}.jpg")
    res = model(img_path, conf=0.5)
    boxes = res[0].boxes.xyxy.tolist()
    cow_number = 0
    for inner_list in boxes:
        cow_number += 1
        x1, y1, x2, y2 = inner_list
        x1, y1, x2, y2 = expand_rectangle_to_square(x1, y1, x2, y2)
        cropped_image = crop_to_square(img_path, x1, y1, x2, y2)
        cropped_image.save(os.path.join(f'{HOME}/output2/', f"frame{number}-{cow_number}.jpg"))